In [ ]:
!pip install kaggle

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download tawsifurrahman/tuberculosis-tb-chest-xray-dataset

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset
License(s): copyright-authors


In [ ]:
!unzip /content/tuberculosis-tb-chest-xray-dataset.zip

Archive:  /content/tuberculosis-tb-chest-xray-dataset.zip
  inflating: TB_Chest_Radiography_Database/Normal.metadata.xlsx  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-10.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-100.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1000.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1001.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1002.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1003.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1004.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1005.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1006.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1007.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1008.png  
  inflating: TB_Chest_Radiography_Database/Normal/Normal-1009.png 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.utils import to_categorical
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


In [ ]:
import tensorflow as tf
img_tensor = tf.convert_to_tensor(test_img, dtype=tf.float32)


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Settings
image_size = (64, 64)
data_path = "/content/TB_Chest_Radiography_Database"
labels = ['Normal', 'Tuberculosis']

X, Y = [], []
for idx, label in enumerate(labels):
    folder = os.path.join(data_path, label)
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, image_size)
            X.append(img)
            Y.append(idx)

X = np.array(X) / 255.0  # Normalize
Y = to_categorical(np.array(Y), num_classes=2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras import layers

def residual_block(x, filters, kernel_size=3, stride=1):
    """Residual block with skip connection."""
    shortcut = x

    # First convolutional layer
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = layers.ReLU()(x)
    x = layers.BatchNormalization()(x)

    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Adjust shortcut dimensions to match x
    shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same')(shortcut)
    shortcut = layers.BatchNormalization()(shortcut)

    # Add shortcut to the output
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_hybrid_model(input_shape=(64, 64, 3), num_classes=2):
    inputs = layers.Input(shape=input_shape)

    # VGG16-like block
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # ResNet-like residual block
    shortcut = x
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.Add()([x, shortcut])
    x = layers.MaxPooling2D((2, 2))(x)

    # Fully connected layers
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the model
model = build_hybrid_model()
model.summary()


Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 64, 64,    │      1,792 │ input_layer_9[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_23    │ (None, 32, 32,    │          0 │ conv2d_48[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 32, 32,    │     73,856 │ max_pooling2d_23… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_24    │ (None, 16, 16,    │          0 │ conv2d_49[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 16, 16,    │    147,584 │ max_pooling2d_24… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 16, 16,    │    147,584 │ conv2d_50[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 16, 16,    │          0 │ conv2d_51[0][0],  │
│                     │ 128)              │            │ max_pooling2d_24… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_25    │ (None, 8, 8, 128) │          0 │ add_6[0][0]       │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 8192)      │          0 │ max_pooling2d_25… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256)       │  2,097,408 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 2)         │        514 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,468,738 (9.42 MB)

 Trainable params: 2,468,738 (9.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model = build_hybrid_cnn()
history = model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 55ms/step - accuracy: 0.8876 - loss: 0.4221 - val_accuracy: 0.8369 - val_loss: 4.2015
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.9588 - loss: 0.1283 - val_accuracy: 0.3679 - val_loss: 1.7004
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9499 - loss: 0.1278 - val_accuracy: 0.9345 - val_loss: 0.1783
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.9755 - loss: 0.0836 - val_accuracy: 0.1631 - val_loss: 54.7699
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.9700 - loss: 0.0975 - val_accuracy: 0.8714 - val_loss: 0.8496
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.9740 - loss: 0.0826 - val_accuracy: 0.6583 - val_loss: 0.9965
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.9784 - loss: 0.0705 - val_accuracy: 0.1631 - val_loss: 22.1882
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9763 - loss: 0.0547 - val_a

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

def apply_gradcam(model, img, class_idx=None):
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer("conv2d_61").output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(img)
        if class_idx is None:
            class_idx = np.argmax(predictions[0])
        class_channel = predictions[:, class_idx]

    grads = tape.gradient(class_channel, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)[0]

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap.numpy(), (img.shape[2], img.shape[1]))
    return heatmap

def overlay_heatmap(heatmap, img, alpha=0.4):
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_VIRIDIS)
    superimposed_img = cv2.addWeighted(heatmap, alpha, img, 1 - alpha, 0)
    return superimposed_img


In [ ]:
# Assuming X_test is your test dataset
test_img = np.expand_dims(X_test[0], axis=0)

# Convert to TensorFlow tensor
test_img_tensor = tf.convert_to_tensor(test_img, dtype=tf.float32)

# Apply Grad-CAM
heatmap = apply_gradcam(model, test_img_tensor)

# Convert to RGB for visualization
img_rgb = np.uint8(test_img[0] * 255)

# Overlay heatmap on the image
superimposed_img = overlay_heatmap(heatmap, img_rgb)

# Display the image
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [ ]:
np_array = test_img_tensor.numpy()

In [ ]:
# Assuming X_test is your test dataset
test_img = np.expand_dims(X_test[0], axis=0)

# Apply Grad-CAM
heatmap = apply_gradcam(model, test_img)

# Convert to RGB for visualization
img_rgb = np.uint8(test_img[0] * 255)
superimposed_img = overlay_heatmap(heatmap, img_rgb)

# Display the image
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor_122']]
Received: inputs=Tensor(shape=(1, 64, 64, 3))
  warnings.warn(msg)


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'